In [1]:
import pandas as pd
import numpy
import json
import statistics as s
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import re
from collections import OrderedDict


In [2]:
header_df = pd.read_json('~/WWData/money_diaries/data/raw_basic_info.json')
diary_df = pd.read_json('~/WWData/money_diaries/data/raw_diaries.json')

In [3]:
header_df = header_df.T

In [7]:
cols = [x for x in header_df.columns if len(x) <= 100 ]
#for col in cols:
    #print(col)


In [8]:
df = header_df.loc[:, header_df.isnull().mean() < .8]

In [9]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(cols)

true_k = 10
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

In [10]:
for i in range(true_k):
    print(f'Cluster {i}'),
    for ind in order_centroids[i, :20]:
        print(f' {terms[ind]}')

Cluster 0
 savings
 membership
 subscription
 water
 payment
 monthly
 card
 pass
 gas
 account
 husband
 expenses
 salary
 loan
 income
 401
 credit
 car
 donation
 electric
Cluster 1
 netflix
 hulu
 hbo
 spotify
 amazon
 prime
 disney
 hbogo
 premium
 subscriptions
 showtime
 espn
 hbonow
 sling
 tv
 stan
 bundle
 plus
 xfinity
 subscription
Cluster 2
 apple
 music
 storage
 icloud
 google
 cloud
 amazon
 spotify
 netflix
 hulu
 care
 prime
 tidal
 itunes
 pandora
 lease
 99
 lessons
 fees
 50gb
Cluster 3
 biweekly
 paycheck
 husband
 partner
 fiancé
 housekeeper
 cleaner
 26
 jobs
 amound
 mortgage
 combined
 cleaning
 house
 1x
 month
 freshly
 fabfitfun
 explain
 express
Cluster 4
 phone
 cell
 internet
 insurance
 car
 netflix
 plan
 utilities
 husband
 wifi
 payment
 fi
 gas
 health
 mobile
 plans
 sprint
 transportation
 daughter
 package
Cluster 5
 internet
 cable
 utilities
 tv
 gas
 electricity
 electric
 wifi
 landline
 cell
 netflix
 phones
 phone
 hbo
 water
 speed
 high


In [11]:
categories = OrderedDict()

categories['income'] = ['paycheck','salary','income']
categories['investments'] = ['401 k','401','401k','roth','ira','savings','investments','retirement']

categories['insurance'] = ['insurance']
categories['health'] = ['hsa','fsa','dental','therapy','medication','disability','vision','medical','health']
categories['home'] =  ['mortgage','renter s insurance','rent','renters','property tax','hoa','housing','home']
categories['utilities'] =  ['internet','wifi','electric','electricity','water','cable','utility','utilities']
categories['car'] = ['auto','fuel','car']
categories['entertainment'] = ['netflix','hulu','hbo','amazon prime','apple','disney','spotify']
categories['debt']  =  ['credit card','student loan','loans','debt']
categories['pet'] = ['cat','dog','pet']



In [12]:
training_data = {}
for col in cols :
    col = re.sub('\W+',' ', col ).lower()
    training_data[col] = None
    for k,v in categories.items():
        for tag in v :
            if re.search(rf"\w?\b{tag}\b\w?",col): 
                training_data[col] = k         
        
    if training_data[col] is None :
        training_data[col] = 'other'


In [13]:
unlabeled_data = { k: v for k,v in training_data.items() if v =='other' }
labeled_data = { k: v for k,v in training_data.items() if v !='other' }

In [14]:
labeled_data

{'salary': 'income',
 'debt': 'debt',
 'paycheck amount 2x month ': 'income',
 'rent': 'home',
 'health dental life disability insurance': 'health',
 'hsa': 'health',
 'car': 'car',
 'spotify': 'entertainment',
 'netflix': 'entertainment',
 'lemonade rental insurance': 'insurance',
 'car insurance': 'car',
 'savings': 'investments',
 'roth ira': 'investments',
 'do you or have you ever received passive or inherited income if yes please explain ': 'income',
 'loans': 'debt',
 'utilities': 'utilities',
 'internet': 'utilities',
 'renter s insurance': 'home',
 'health insurance': 'health',
 'my salary': 'income',
 'my wife s salary': 'income',
 'my paycheck': 'income',
 'my wife s paycheck': 'income',
 'mortgage': 'home',
 'student loan': 'debt',
 'cable internet': 'utilities',
 'life insurance': 'insurance',
 'home warranty': 'home',
 'credit card': 'debt',
 'my husband s salary': 'income',
 'my paycheck biweekly ': 'income',
 'husband s paycheck amount biweekly ': 'income',
 'hulu': 'en